### Testing optimization techniques of geodata handling using 3D buildings from Yangzi Che et al.

There is a global ML retrieved 3D building footprint dataset available from the Copernicus Global Land Service: 
3D-GloBFP: the first global three-dimensional building footprint dataset
https://essd.copernicus.org/articles/16/5357/2024/essd-16-5357-2024-assets.html

In [39]:
## Import libraries
# system
import os
import time
import multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging

# data manipulation
import duckdb
import polars as pl
import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import xarray as xr
from rasterio.transform import from_bounds
from rasterio.features import geometry_mask, rasterize
import shapely
from shapely.geometry import box, Polygon
from shapely import wkb
from shapely import get_coordinates
import pyarrow as pa
import pyarrow.feather as feather
import pyarrow.parquet as pq
from lonboard import viz

# visualization
from rasterio.plot import show
import folium
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

repo_name = 'masterthesis_genai_spatialplan'
if not repo_name in os.getcwd():
    os.chdir(repo_name)

p=os.popen('git rev-parse --show-toplevel')
repo_dir = p.read().strip()
p.close()

#seaborn cmap
rocket=sns.color_palette("rocket", as_cmap=True)
mako=sns.color_palette("mako", as_cmap=True)

In [40]:
# Define bbox for entire Leipzig
xmin, ymin = 12.2714, 51.2705  # Southwest corner
xmax, ymax = 12.5019, 51.4174  # Northeast corner

# Create bbox polygon
bbox = box(xmin, ymin, xmax, ymax)

# Create GeoDataFrame
bbox_gdf = gpd.GeoDataFrame(geometry=[bbox], crs="EPSG:4326")

bbox_gdf.explore()

### Experimenting with DuckDB 

In [3]:
# Install and load the spatial extension
duckdb.sql("""
    INSTALL spatial;
    LOAD spatial;
    SET enable_geoparquet_conversion = false;
""")

This dataset is huge! So reading it using geopandas will take hours. Using duckdb and polars is faster, but still slow. E.g. the following is possible 

::: {.callout-tip}
skip this cell! for faster reading use the Docker-GDAL conversion method below

:::

In [ ]:
query = f"""
    SELECT * 
    FROM ST_Read('{repo_dir}/data/che_etal/Germany_Hungary_Iceland/Germany.shp') 
    WHERE ST_Within(geom, ST_MakeEnvelope(CAST({xmin} AS DOUBLE), CAST({ymin} AS DOUBLE), 
                                          CAST({xmax} AS DOUBLE), CAST({ymax} AS DOUBLE)));
"""
# COPY (query) TO 'building_heights_germany.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON');
#building_heights_gdf=gpd.read_file(f"{repo_dir}/data/che_etal/building_heights_germany.geojson")

# Install and load the spatial extension
duckdb.sql("""INSTALL spatial;
LOAD spatial;""")

building_heights_gdf=duckdb.sql(query).pl().to_pandas()

display(building_heights_gdf)

building_heights_gdf=building_heights_gdf.assign(geometry=building_heights_gdf.geom.apply(wkb.loads))

# save to parquet for ease of use
building_heights_gdf.to_parquet(f"{repo_dir}/data/che_etal/building_heights_germany.parquet", index=False)

```bash

docker pull osgeo/gdal:ubuntu-full-3.5.2
docker run --rm -it -v ${PWD}/data/che_etal/Germany_Hungary_Iceland:/data osgeo/gdal:ubuntu-full-3.5.2 `
  ogr2ogr `
  /data/building_heights_germany.parquet `
  /data/Germany.shp `
  -dialect SQLite `
  -sql "SELECT geometry, Height FROM 'Germany'" `
  -lco COMPRESSION=BROTLI `
  -lco POLYGON_ORIENTATION=COUNTERCLOCKWISE `
  -lco ROW_GROUP_SIZE=9999999

```

In [5]:
# check parquet schema
schema_query = f"""
    SELECT * FROM parquet_schema('{repo_dir}/data/che_etal/Germany_Hungary_Iceland/building_heights_germany.parquet')
"""
schema = duckdb.sql(schema_query).pl()
schema

file_name,name,type,type_length,repetition_type,num_children,converted_type,scale,precision,field_id,logical_type
str,str,str,str,str,i64,str,i64,i64,i64,str
"""C:/Users/janne/Dropbox/Mein PC…","""schema""",null,null,"""REQUIRED""",2,null,null,null,null,null
"""C:/Users/janne/Dropbox/Mein PC…","""Height""","""DOUBLE""",null,"""OPTIONAL""",null,null,null,null,null,null
"""C:/Users/janne/Dropbox/Mein PC…","""GEOMETRY""","""BYTE_ARRAY""",null,"""OPTIONAL""",null,null,null,null,null,null


In [ ]:
query = f"""
COPY (
  SELECT
    Height AS height,
    ST_GeomFromWKB("GEOMETRY") AS geometry
  FROM read_parquet('{repo_dir}/data/che_etal/Germany_Hungary_Iceland/building_heights_germany.parquet', filename=true, hive_partitioning=1)
  ORDER BY ST_Hilbert(ST_GeomFromWKB("GEOMETRY"), ST_Extent(ST_MakeEnvelope({xmin}, {ymin}, {xmax}, {ymax})))
)
TO '{repo_dir}/data/che_etal/building_heights_germany_sorted.parquet'
WITH (FORMAT PARQUET, COMPRESSION 'zstd');
"""


# Install and load the spatial extension
duckdb.sql("""
    INSTALL spatial;
    LOAD spatial;
    SET enable_geoparquet_conversion = false;
""")

duckdb.sql(query)

In [4]:
drivers=duckdb.sql("SELECT * FROM ST_Drivers();").pl()
display(drivers)

#get list of short names of drivers
drivers.select(pl.col("short_name")).to_series().to_list()

short_name,long_name,can_create,can_copy,can_open,help_url
str,str,bool,bool,bool,str
"""ESRI Shapefile""","""ESRI Shapefile""",true,false,true,"""https://gdal.org/drivers/vecto…"
"""MapInfo File""","""MapInfo File""",true,false,true,"""https://gdal.org/drivers/vecto…"
"""UK .NTF""","""UK .NTF""",false,false,true,"""https://gdal.org/drivers/vecto…"
"""LVBAG""","""Kadaster LV BAG Extract 2.0""",false,false,true,"""https://gdal.org/drivers/vecto…"
"""S57""","""IHO S-57 (ENC)""",true,false,true,"""https://gdal.org/drivers/vecto…"
…,…,…,…,…,…
"""PMTiles""","""ProtoMap Tiles""",true,false,true,"""https://gdal.org/drivers/vecto…"
"""JSONFG""","""OGC Features and Geometries JS…",true,false,true,"""https://gdal.org/drivers/vecto…"
"""TIGER""","""U.S. Census TIGER/Line""",false,false,true,"""https://gdal.org/drivers/vecto…"


['ESRI Shapefile',
 'MapInfo File',
 'UK .NTF',
 'LVBAG',
 'S57',
 'DGN',
 'OGR_VRT',
 'Memory',
 'CSV',
 'GML',
 'GPX',
 'KML',
 'GeoJSON',
 'GeoJSONSeq',
 'ESRIJSON',
 'TopoJSON',
 'OGR_GMT',
 'GPKG',
 'SQLite',
 'WAsP',
 'OpenFileGDB',
 'DXF',
 'CAD',
 'FlatGeobuf',
 'Geoconcept',
 'GeoRSS',
 'VFK',
 'PGDUMP',
 'OSM',
 'GPSBabel',
 'WFS',
 'OAPIF',
 'EDIGEO',
 'SVG',
 'ODS',
 'XLSX',
 'Elasticsearch',
 'Carto',
 'AmigoCloud',
 'SXF',
 'Selafin',
 'JML',
 'PLSCENES',
 'CSW',
 'VDV',
 'MVT',
 'NGW',
 'MapML',
 'GTFS',
 'PMTiles',
 'JSONFG',
 'TIGER',
 'AVCBin',
 'AVCE00']

In [ ]:
query = f"""
COPY (
  SELECT
    Height AS height,
    ST_GeomFromWKB("GEOMETRY") AS geometry
  FROM read_parquet('{repo_dir}/data/che_etal/Germany_Hungary_Iceland/building_heights_germany.parquet', filename=true, hive_partitioning=1)
  ORDER BY ST_Hilbert(ST_GeomFromWKB("GEOMETRY"), ST_Extent(ST_MakeEnvelope({xmin}, {ymin}, {xmax}, {ymax})))
)
TO '{repo_dir}/data/che_etal/building_heights_germany_sorted.parquet'
WITH (
  FORMAT 'GDAL', 
  DRIVER 'FlatGeobuf',
  LAYER_CREATION_OPTIONS ['OVERWRITE=YES', 'HILBERT_ORDER=2', 'HILBERT_EXTENT=ST_MakeEnvelope({xmin}, {ymin}, {xmax}, {ymax})'],
);
"""

In [ ]:
query = f"""
    SELECT
       Height AS height,
       ST_GeomFromWKB("GEOMETRY") AS geom,                 
    FROM read_parquet('{repo_dir}/data/che_etal/Germany_Hungary_Iceland/building_heights_germany.parquet', filename=true, hive_partitioning=1)
    WHERE ST_Within(
        ST_GeomFromWKB("GEOMETRY"),
        ST_MakeEnvelope({xmin}, {ymin}, {xmax}, {ymax})
    )
"""
# COPY (query) TO 'building_heights_germany.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON');
#building_heights_gdf=gpd.read_file(f"{repo_dir}/data/che_etal/building_heights_germany.geojson")



# Install and load the spatial extension
duckdb.sql("""
    INSTALL spatial;
    LOAD spatial;
    SET enable_geoparquet_conversion = false;
""")

building_heights_gdf=duckdb.sql(query).pl().to_pandas()

display(building_heights_gdf)

In [ ]:
duckdb.sql("SELECT 42").fetchall()   # Python objects
duckdb.sql("SELECT 42").df()         # Pandas DataFrame
duckdb.sql("SELECT 42").pl()         # Polars DataFrame
duckdb.sql("SELECT 42").arrow()      # Arrow Table
duckdb.sql("SELECT 42").fetchnumpy() # NumPy Arrays

In [ ]:
def construct_arrow_table(df: pd.DataFrame, geometry) -> pa.Table:
    # Note in this quick example we omit metadata on the table header
    table = pa.Table.from_pandas(df)
    # coords = pygeos.get_coordinates(geometry)
    coords = get_coordinates(geometry)
    parr = pa.FixedSizeListArray.from_arrays(coords.flat, 2)
    return table.append_column("geometry", parr)



In [ ]:
query = f"""
    SELECT
       Height AS height,
       ST_AsWKB(ST_GeomFromWKB("GEOMETRY")) AS geom                 
    FROM read_parquet('{repo_dir}/data/che_etal/Germany_Hungary_Iceland/building_heights_germany.parquet', filename=true, hive_partitioning=1)
    WHERE ST_Within(
        ST_GeomFromWKB("GEOMETRY"),
        ST_MakeEnvelope({xmin}, {ymin}, {xmax}, {ymax})
    ) 
"""

# Install and load the spatial extension
duckdb.sql("""
    INSTALL spatial;
    LOAD spatial;
    SET enable_geoparquet_conversion = false;
""")

building_heights_db = duckdb.sql(query)
building_heights_table = building_heights_db.fetch_arrow_table()

In [ ]:
from lonboard import PolygonLayer

con=duckdb.connect()
layer = PolygonLayer.from_duckdb(
    building_heights_db,
    con=con,
    get_fill_color=[255, 0, 0],
)

In [ ]:
query = f"""
    SELECT
       Height AS height,
       ST_AsWKB(ST_GeomFromWKB("GEOMETRY")) AS geom                 
    FROM read_parquet('{repo_dir}/data/che_etal/Germany_Hungary_Iceland/building_heights_germany.parquet', filename=true, hive_partitioning=1)
    WHERE ST_Within(
        ST_GeomFromWKB("GEOMETRY"),
        ST_MakeEnvelope({xmin}, {ymin}, {xmax}, {ymax})
    ) 
"""

# Install and load the spatial extension
duckdb.sql("""
    INSTALL spatial;
    LOAD spatial;
    SET enable_geoparquet_conversion = false;
""")

building_heights_db = duckdb.sql(query)

In [ ]:
df= building_heights_db.fetchdf()
df

In [ ]:
# query within bbox and sort by hilbert curve
query = f"""
    SELECT
       Height AS height,
       ST_AsWKB(ST_GeomFromWKB("GEOMETRY")) AS geom                 
    FROM read_parquet('{repo_dir}/data/che_etal/Germany_Hungary_Iceland/building_heights_germany.parquet', filename=true, hive_partitioning=1)
    WHERE ST_Within(
        ST_GeomFromWKB("GEOMETRY"),
        ST_MakeEnvelope({xmin}, {ymin}, {xmax}, {ymax})
    )
    ORDER BY ST_Hilbert(ST_GeomFromWKB("GEOMETRY"), ST_Extent(ST_MakeEnvelope({xmin}, {ymin}, {xmax}, {ymax})))
"""

# Install and load the spatial extension
duckdb.sql("""
    INSTALL spatial;
    LOAD spatial;
    SET enable_geoparquet_conversion = false;
""")

building_heights_db = duckdb.sql(query)
building_heights_table = building_heights_db.fetch_arrow_table()
building_heights_df= building_heights_db.fetchdf()

This works and selects efficiently and spatially sorted to geodataframe:

In [ ]:
# query within bbox and sort by hilbert curve
duckdb.sql(f"""
    CREATE TEMP TABLE tmp_buildings AS
    SELECT
        Height AS height,
        ST_AsWKB(ST_GeomFromWKB("GEOMETRY")) AS geom                 
    FROM read_parquet('{repo_dir}/data/che_etal/Germany_Hungary_Iceland/building_heights_germany.parquet', filename=true, hive_partitioning=1)
    WHERE ST_Within(
        ST_GeomFromWKB("GEOMETRY"),
        ST_MakeEnvelope({xmin}, {ymin}, {xmax}, {ymax})
    )
    ORDER BY ST_Hilbert(ST_GeomFromWKB("GEOMETRY"), ST_Extent(ST_MakeEnvelope({xmin}, {ymin}, {xmax}, {ymax})))
""")


In [ ]:
# fetch as Arrow table
building_heights_table = duckdb.sql("SELECT * FROM tmp_buildings").arrow()

# fetch as pandas df
building_heights_df = duckdb.sql("SELECT * FROM tmp_buildings").df()

In [ ]:
wkb_list = building_heights_table['geom'].to_pylist()

# from shapely import to_wkt
# print(to_wkt(wkb.loads(wkb_list[0])))

# collect coordinates
geom_offsets = [0]
ring_offsets = []
xs = []
ys = []

for wkb_blob in wkb_list:
    geom = wkb.loads(wkb_blob)
    
    if geom.geom_type == 'Polygon':
        rings = [geom.exterior] + list(geom.interiors)
    elif geom.geom_type == 'MultiPolygon':
        rings = []
        for poly in geom.geoms:
            rings.append(poly.exterior)
            rings.extend(poly.interiors)
    else:
        continue  # skip non-polygonal types

    # Add each ring
    for ring in rings:
        ring_coords = np.array(ring.coords)
        xs.extend(ring_coords[:, 0])
        ys.extend(ring_coords[:, 1])
        ring_offsets.append(len(xs))

    geom_offsets.append(len(ring_offsets))

# Convert to numpy arrays
xs = np.array(xs)
ys = np.array(ys)
ring_offsets = np.array(ring_offsets, dtype=np.int32)
geom_offsets = np.array(geom_offsets[:-1], dtype=np.int32)

# build geoarrow polygon array
polygon_array = ga.polygon().from_geobuffers(
    None,            # no validity bitmap
    geom_offsets,    # polygon offset
    ring_offsets,    # ring offset
    xs,
    ys
)
gdf= ga.to_geopandas(polygon_array)

building_heights_gdf=gpd.GeoDataFrame(building_heights_df, geometry=gdf.geometry, crs="EPSG:4326")
building_heights_gdf = building_heights_gdf.drop(columns=['geom'])
building_heights_gdf

### Geopandas optimization

In [ ]:
# read the sorted parquet file 
gdf=gpd.read_parquet(f"{repo_dir}/data/che_etal/Germany_Hungary_Iceland/building_heights_germany.parquet")#, bbox=(xmin, ymin, xmax, ymax))

In [ ]:
viz(gdf[0:1000])